In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pandas as pd
import numpy as np
import re
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import Embedding
from keras.layers import Bidirectional
from tensorflow.keras.preprocessing.text import one_hot
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
# Load data
df = pd.read_csv('./data/product_df.csv')
df = df[['Star Rating', 'Comment']]

In [3]:
def label_sentiment(rating):
    if rating in [1, 2]:
        return '0'
    elif rating == 3:
        return '1'
    elif rating in [4, 5]:
        return '2'
    else:
        return '3'  # Nếu có xếp hạng nằm ngoài khoảng 1-5

# Gắn nhãn cảm xúc cho mỗi đánh giá
df['Sentiment'] = df['Star Rating'].apply(label_sentiment)

# Hiển thị 5 hàng đầu tiên của dataframe với cột sentiment mới
print(df.head())

   Star Rating                                            Comment Sentiment
0            4                      Điện thoại này dùng rất thích         2
1            4                               sử dụng thấy cũng ok         2
2            2                      Bảo hành ít quá, chỉ 12 tháng         0
3            5                             Sản phẩm mượt, chạy êm         2
4            3  Cho mình hỏi muốn khởi động lại máy hay tắt ng...         1


In [4]:
# Kiểm tra và loại bỏ giá trị khuyết thiếu
print(df.isnull().sum())
df = df.dropna(subset=['Comment'])
# Kiểm tra và loại bỏ dữ liệu trùng lặp
duplicate_comments = df[df.duplicated(['Comment'])]
print("Các dòng dữ liệu trùng lặp trong cột 'Comment':")
print(duplicate_comments)
df = df.drop_duplicates(['Comment'])
print("Shape after dropping duplicates:", df.shape)

Star Rating    0
Comment        0
Sentiment      0
dtype: int64
Các dòng dữ liệu trùng lặp trong cột 'Comment':
      Star Rating                                   Comment Sentiment
48              4                                   Rất tốt         2
76              5                                    Rất ok         2
107             2                             pin tụt nhanh         0
135             5                              sản phẩm tốt         2
173             2                             hao pin nhanh         0
177             5                              sản phẩm tốt         2
181             5                                    Rất ok         2
183             5                              sản phẩm tốt         2
192             5                               sản phẩm ok         2
205             5                               máy dùng ok         2
225             4                                       Tốt         2
227             5                               

In [5]:
# Chuẩn hóa và làm sạch văn bản
def remove_special_characters(text):
    # Loại bỏ các ký tự đặc biệt, giữ lại chữ cái, số, và các dấu câu
    return re.sub(r'[^a-zA-ZÀ-ỹà-ỹ0-9\s]', '', text)

def to_lowercase(text):
    # Chuyển đổi văn bản về chữ thường
    return text.lower()

def normalize_text(text):
    text = remove_special_characters(text)
    text = to_lowercase(text)
    return text

df['Comment'] = df['Comment'].apply(normalize_text)
print(df.head())

   Star Rating                                            Comment Sentiment
0            4                      điện thoại này dùng rất thích         2
1            4                               sử dụng thấy cũng ok         2
2            2                       bảo hành ít quá chỉ 12 tháng         0
3            5                              sản phẩm mượt chạy êm         2
4            3  cho mình hỏi muốn khởi động lại máy hay tắt ng...         1


In [6]:
max_features = 20000 
maxlen = 200

In [7]:
phrases = df['Comment'].tolist()
vocab_size = 50000
encoded_phrases = [one_hot(d, vocab_size) for d in phrases]
df['Comment'] = encoded_phrases

In [8]:
label_encoder = LabelEncoder()
df['Sentiment'] = label_encoder.fit_transform(df['Sentiment'])

In [9]:
# Chia dữ liệu thành đặc trưng (X) và nhãn (y)
X = df['Comment']
y = df['Sentiment']

In [10]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [11]:
x_train = keras.preprocessing.sequence.pad_sequences(X_train, maxlen=maxlen)
x_val = keras.preprocessing.sequence.pad_sequences(X_val, maxlen=maxlen)

In [12]:
model = Sequential()
inputs = keras.Input(shape=(None,), dtype="int32")
# Embed each integer in a 128-dimensional vector
model.add(inputs)
model.add(Embedding(50000, 128))
# Add 2 bidirectional LSTMs
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Bidirectional(LSTM(64)))
# Add a classifier
model.add(Dense(3, activation="sigmoid"))
#model = keras.Model(inputs, outputs)
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, None, 128)      │     6,400,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, None, 128)      │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,598,019 (25.17 MB)

 Trainable params: 6,598,019 (25.17 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=32, epochs=30, validation_data=(x_val, y_val))

Epoch 1/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 32s 483ms/step - accuracy: 0.5423 - loss: 0.9669 - val_accuracy: 0.6367 - val_loss: 0.8211
Epoch 2/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 19s 444ms/step - accuracy: 0.7045 - loss: 0.6451 - val_accuracy: 0.7267 - val_loss: 0.6567
Epoch 3/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 17s 446ms/step - accuracy: 0.8124 - loss: 0.4386 - val_accuracy: 0.7700 - val_loss: 0.6057
Epoch 4/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 17s 437ms/step - accuracy: 0.9034 - loss: 0.2837 - val_accuracy: 0.7633 - val_loss: 0.7682
Epoch 5/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 16s 413ms/step - accuracy: 0.9468 - loss: 0.1788 - val_accuracy: 0.7667 - val_loss: 0.7657
Epoch 6/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 15s 398ms/step - accuracy: 0.9710 - loss: 0.1061 - val_accuracy: 0.7700 - val_loss: 0.8344
Epoch 7/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 18s 470ms/step - accuracy: 0.9801 - loss: 0.0775 - val_accuracy: 0.7633 - val_loss: 0.9649
Epoch 8/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 16s 413ms/step - accuracy: 0.9770 - loss: 0.0721 - val_accu

In [14]:
model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_val, y_val))

Epoch 1/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 17s 438ms/step - accuracy: 0.9994 - loss: 0.0062 - val_accuracy: 0.7400 - val_loss: 1.5736
Epoch 2/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 15s 396ms/step - accuracy: 0.9971 - loss: 0.0073 - val_accuracy: 0.7300 - val_loss: 1.5807
Epoch 3/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 15s 400ms/step - accuracy: 0.9995 - loss: 0.0020 - val_accuracy: 0.7300 - val_loss: 1.6475
Epoch 4/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 15s 385ms/step - accuracy: 0.9985 - loss: 0.0055 - val_accuracy: 0.7433 - val_loss: 1.7131
Epoch 5/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 15s 389ms/step - accuracy: 1.0000 - loss: 0.0021 - val_accuracy: 0.7400 - val_loss: 1.7699


In [23]:
# Các thông số đã sử dụng trong quá trình huấn luyện
vocab_size = 50000
maxlen = 200
# Chuỗi đánh giá mới cần dự đoán
new_comment = "Quá tuyệt vời, con gái tôi rất thích"

# Chuẩn hóa và mã hóa văn bản
normalized_comment = normalize_text(new_comment)
encoded_comment = one_hot(normalized_comment, vocab_size)
padded_comment = pad_sequences([encoded_comment], maxlen=maxlen)

# Dự đoán
prediction = model.predict(padded_comment)
sentiment_label = np.argmax(prediction)

# Gắn nhãn cảm xúc
if sentiment_label == 0:
    print('Cảm xúc: Tiêu cực')
elif sentiment_label == 1:
    print('Cảm xúc: Trung tính')
elif sentiment_label == 2:
    print('Cảm xúc: Tích cực')
else:
    print('Không xác định')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
Cảm xúc: Tích cực


In [16]:
# Đánh giá mô hình trên tập kiểm tra
_, accuracy = model.evaluate(x_val, y_val)
print(f'Accuracy trên tập kiểm tra: {accuracy:.2f}')


10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 109ms/step - accuracy: 0.7277 - loss: 1.8056
Accuracy trên tập kiểm tra: 0.74


In [17]:
from sklearn.metrics import classification_report, confusion_matrix

# Dự đoán trên tập kiểm tra
y_pred = model.predict(x_val)
y_pred_labels = np.argmax(y_pred, axis=1)

# Đánh giá mô hình
print(classification_report(y_val, y_pred_labels))


10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 98ms/step
              precision    recall  f1-score   support

           0       0.70      0.83      0.76        86
           1       0.36      0.33      0.35        48
           2       0.88      0.81      0.84       166

    accuracy                           0.74       300
   macro avg       0.65      0.66      0.65       300
weighted avg       0.74      0.74      0.74       300



In [18]:
model.save('sentiment_model.h5')